In [7]:
import numpy as np 
import matplotlib.pyplot as plt
import nengo

# Hannas Test Net

In [8]:
net = nengo.Network()

In [ ]:
with net:
    node1 = nengo.Node(output=0.5) # use output for input layer, poisson spike function with frequency = pixel intensity
    neurons = []
    for n in range(5):
        neurons.append(nengo.Ensemble(1,2))